<img src=".\img\mioti.png">   



# Proyecto Reconocimiento Facial: Detector de Caras Flash

<img src="./img/emociones.png" style="width: 800px">

### Objetivos:
Este notebook es necesario para el POC de la solución Móvil. Utiliza Flask para levantar un servidor en local y así se puede consultar una página HTML desde el dispositivo móvil, desde reconocer a la persona y sus emociones

In [16]:
#pip install flask

In [17]:
from flask import Flask
from flask import render_template
from flask import Response
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

In [18]:
app=Flask(__name__)

In [19]:
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [20]:
file_model="./model/vgg-face-model-v4.json"
file_weights="./model/vgg-face-v4.h5"
class_labels=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [21]:
json_file = open(file_model, 'r')
model_json = json_file.read()
json_file.close()
model = tf.keras.models.model_from_json(model_json)
model.load_weights(file_weights)

In [22]:
%%time
reconocedor_facial=cv2.face.EigenFaceRecognizer_create()
reconocedor_facial.read('./model/modelo_reconocimiento_caras.xml')

Wall time: 49.7 s


In [23]:
data_path='./reconocimiento'
lista_gente=os.listdir(data_path)
print(lista_gente)

['Javier', 'Juan Pedro', 'Ricardo']


In [24]:
def draw_text_with_backgroud(img, text, x, y, font_scale, thickness=1, font=cv2.FONT_HERSHEY_SIMPLEX,
                            background=(175,50,200), foreground=(255,255,255), box_coords_1=(-5,5), box_coords_2=(5,-5)):
    (text_width, text_height) = cv2.getTextSize(text, font, fontScale=font_scale, thickness=1)[0]
    box_coords = ((x+box_coords_1[0], y+box_coords_1[1]), (x + text_width + box_coords_2[0], y - text_height + box_coords_2[1]))
    cv2.rectangle(img, box_coords[0], box_coords[1], background, cv2.FILLED)
    cv2.putText(img, text, (x, y), font, fontScale=font_scale, color=foreground, thickness=thickness)

In [25]:
def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.2, 5)
    if len(faces) == 0:
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    for idx,face in enumerate(faces):
        x,y,w,h = face
    
        cv2.rectangle(img,(x,y),(x+w,y+h),(175,50,200),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation = cv2.INTER_CUBIC)
        roi_gray_face = cv2.resize(roi_gray, (150, 150), interpolation = cv2.INTER_CUBIC)
        
        if np.sum([roi_gray]) != 0.0:
            roi = roi_gray.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

        # make a prediction on the ROI, then lookup the class
        
        #para hacer la prediccion hay que cambiar el tamaño a la imagen
        
            #Predicción de persona
            result=reconocedor_facial.predict(roi_gray_face)
            
            #Predicción de emoción
            preds = model.predict(roi)[0]
            
            if result[1]<10000:
                label = f'Persona {lista_gente[result[0]]} EMOCION {class_labels[preds.argmax()]}'  
            else:
                label = f'Persona Desconocido EMOCION {class_labels[preds.argmax()]}'

            draw_text_with_backgroud(img, label, x + 5, y, font_scale=0.4)
            if (idx==1):
                draw_text_with_backgroud(img, label, x + 5, y, font_scale=0.4, background=(15,150,200), foreground=(255,255,255), box_coords_1=(-7,7), box_coords_2=(7,-7))
        else:
            draw_text_with_backgroud(img, "No Face Found", x + 5, y, font_scale=0.4)
   
            
    return (x,w,y,h), roi_gray, img

In [26]:
cap = cv2.VideoCapture(1)

In [27]:
def generate():
    while True:

        ret, frame = cap.read()
        rect, face, image = face_detector(frame)

    
        res = cv2.resize(image, dsize=(800,600),interpolation=cv2.INTER_CUBIC)
        cv2.namedWindow("Reconocedor de Emociones", cv2.WINDOW_NORMAL)
        cv2.imshow("Reconocedor de Emociones", res)
        
        #Se comprime imagen y se almacena en memoria
        (flag,encodedImage)=cv2.imencode('.jpg',frame)
        #Si la imagen no fué codificada con éxito
        if not flag:
            continue
        yield(b'--frame\r\n' b'Content-Type:image/jpeg\r\n\r\n'+bytearray(encodedImage)+b'\r\n')

In [28]:
@app.route("/")

def index():
    return render_template("index.html")

In [29]:
@app.route("/video_feed")

def video_feed():
    return Response(generate(),mimetype='multipart/x-mixed-replace;boundary=frame')

In [30]:
if __name__=="__main__":
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jan/2022 21:12:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 21:12:55] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 21:13:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 21:13:25] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 21:13:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Jan/2022 21:14:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 21:14:50] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 21:16:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2022 21:16:22] "GET /video_feed HTTP/1.1" 200 -


In [ ]:
cap.relase()